In [ ]:
# VGGNet중 가장 간단한 모델인 VGG11을 구현할 예정이다.
# 필요한 모듈 import

import copy
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as Datasets

In [ ]:
# VCG모델을 정의하는 클래스 제작
# 입력의 feature에 따라서 VGG11, VGG13, VGG16, VGG19가 될 수 있다.

class VGG(nn.Module):
  def __init__(self, features, output_dim):
    super(VGG, self).__init__()
    self.features = features
    self.avgpool = nn.AdaptiveAvgPool2d(7)
    self.classifier = nn.Sequential(
        nn.Linear(512*7*7, 4096),
        nn.ReLU(inplace = True),
        nn.Dropout(0.5),
        nn.Linear(4096,4096),
        nn.ReLU(inplace = True),
        nn.Dropout(0.5),
        nn.Linear(4096, output_dim)
    )

  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    h = x.view(x.shape[0],-1)
    x = self.classifier(h)
    return x, h

In [ ]:
# 각 VGG모델의 계층들을 정리한 것이다
# M : Max Pooling을 의미한다
# 숫자 : 입력 데이터를 해당 숫자의 채널값을 가진 feature map으로 변환해주는 Convolutional Layer를 의미한다

vgg11_config = [64,'M',128,'M',256,256,'M',512,512,'M',512,512,'M']
vgg13_config = [64,64,'M',128,128,'M',256,256,'M',512,512,'M',512,512,'M']
vgg16_config = [64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M']
vgg19_config = [64,64,'M',128,128,'M',256,256,256,256,'M',512,512,512,512,'M',512,512,512,512,'M']

In [ ]:
# VGG에서 convolution layer를 정의하기
def get_vgg_layers(config, batch_norm):
  layers = []
  in_channels = 3

  for c in config:
    assert c == "M" or isinstance(c, int)
    
    # 만약 config에서 추출된 원소가 'M'이라면 shape를 반으로 감소시키는 최대풀링층을 추가한다 
    if c == "M":
      layers += [nn.MaxPool2d(kernel_size = 2)]
    else:
      conv2d = nn.Conv2d(in_channels, c, kernel_size=3, padding=1)

      # 배치 정규화를 사용하도록 설정이 되어 있다면
      # convolutional layer와 ReLU계층 사이에 Batch Normalization layer를 함께 추가시키도록 한다
      if batch_norm:
        layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace=True)]
      else:
        layers += [conv2d, nn.ReLU(inplace=True)]
      in_channels = c

  return nn.Sequential(*layers)    

In [ ]:
# VGG11 model의 앞부분을 생성한다.
# 물론 해당 model은 VGG Network의 앞 부분인 convolution 계층에 해당한다
# 후에 VGG클래스의 생성자에 이 모델을 삽입하여 완성된 VGG11모델을 반환할 것이다.
vgg11_layers = get_vgg_layers(vgg11_config, batch_norm = True)

In [ ]:
# 완성된 VGG11네트워크 생성하기
OUTPUT_DIM = 2 # 개와 고양이 두 개의 클래스를 이용하므로
model = VGG(vgg11_layers, OUTPUT_DIM)
print(model)

# 사전 훈련된 모델을 다음과 같이 생성해도 되지만, 직접 구현하기 위해 다음 방법을 사용하지는 않았다.
# import torchvision.models as models
# pretrained_model = models.vgg11_bn(pretrained = True)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [ ]:
# 모델을 완결하였으니 본격적으로 이미지에 대한 전처리를 위한 코드를 작성하자
train_transforms = transforms.Compose([
                  transforms.Resize((256,256)),
                  transforms.RandomRotation(5), # 주어진 이미지 데이터를 5 degree 이하로 회전시키는 data augmentation
                  transforms.RandomHorizontalFlip(0.5),
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

test_transforms = transforms.Compose([
                  transforms.Resize((256,256)),
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [ ]:
# 데이터셋을 정리하기
# train 디렉토리와 test 디렉토리가 존재한다
# train, test 디렉토리 각각에 Cat, Dog 디렉토리가 있다.

train_path = "train"
test_path = "test"

train_dataset = torchvision.datasets.ImageFolder(train_path, transform = train_transforms)
test_dataset = torchvision.datasets.ImageFolder(test_path, transform = test_transforms)

print(len(train_dataset))

In [ ]:
# 훈련데이터와 검증데이터를 9대 1의 비율로 분할한다
# 그리고 검증데이터를 미리 test_transforms로 전처리 한다.

VALID_RATIO = 0.9
n_train_examples = int(len(train_dataset) * VALID_RATIO)
n_valid_examples = len(train_dataset) - n_train_examples

train_data, valid_data = data.random_split(train_dataset, [n_train_examples, n_valid_examples])

valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

['.config', 'test', '.ipynb_checkpoints', 'train', 'sample_data']

In [ ]:
# 데이터로더를 이용하여 데이터셋의 데이터를 메모리로 가져온다
BATCH_SIZE = 128

train_iterator = data.DataLoader(train_data, shuffle = True, batch_size = BATCH_SIZE)
valid_iterator = data.DataLoader(valid_data, shuffle = True, batch_size = BATCH_SIZE)
test_iterator = data.DataLoader(test_dataset, shuffle = True, batch_size = BATCH_SIZE)

In [ ]:
# 모델학습에서 사용할 optimizer와 손실함수를 정의하기
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr = 1e-7)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
# 모델의 정확도를 측정하기 위한 함수를 정의하기
def calculate_accuracy(y_pred, y):
  top_pred = y_pred.argmax(1, keepdim=True)
  correct = top_pred.eq(y.view_as(top_pred)).sum() # 총 정답 개수
  acc = correct.float() / y.shape[0] # 총 정답 개수를 배치 개수만큼 나누어준다
  return acc

In [ ]:
# 모델 학습을 위한 함수를 정의하기
# 1 epoch를 훈련하는 함수이다
def train(model, iterator, optimizer, criterion, device):
  
  epoch_loss = 0
  epoch_acc = 0
  
  model.train()
  # train_dataloader로부터 한번에 배치 개수만큼 데이터를 불러오는 루틴이다. 
  for (x, y) in iterator:
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad()
    y_pred, _ = model(x) # 아까 모델에서 2개의 값을 반환한다고 정의하였었다. 각 클래스별 확률값은 첫 번째 반환값으로 설정하였다.
    loss = criterion(y_pred, y)
    acc = calculate_accuracy(y_pred, y)
    loss.backward()
    optimizer.step()

    # 평균 정확도와 평균 손실함수값을 축적해나간다
    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# 모델의 성능을 측정하는 함수
def evaluate(model, iterator, criterion, device):
  epoch_loss = 0
  epoch_acc = 0

  model.eval()
  with torch.no_grad():
    for (x, y) in iterator:
      x, y = x.to(device), y.to(device)
      y_pred, _ = model(x)
      loss = criterion(y_pred, y)
      acc = calculate_accuracy(y_pred, y)
      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# 모델의 학습시간을 측정하기 위한 함수
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [ ]:
# 모델을 학습
EPOCHS = 5
best_valid_loss = float('inf')
for epoch in range(EPOCHS):
  start_time = time.monotonic() # 현재 OS에서의 시간값 반환
  train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(),"./best_VGG_model.pt")

  end_time = time.monotonic()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f"EPOCH : {epoch+1} | Epoch Time : {epoch_mins}m {epoch_secs}s")
  print(f"Train Loss : {train_loss} | Train Acc : {train_acc}")
  print(f"Valid Loss : {valid_loss} | Valid Acc : {valid_acc}")


In [ ]:
# 앞서 저장했던 손실함수값이 가장 낮았던 모델을 불러와서 Test dataset으로 성능을 측정해보자
model.load_state_dict("./best_VGG_model.pt")
test_loss, test_acc = evaluate(model, test_iterator, criterion, device)
print(f"Test Loss : {test_loss} | Test Acc : {test_acc}")


# Test데이터셋을 이용한 모델 성능 측정 결과
# Test loss는 0.684
# Test Acc는 0.5로 결과가 좋지 않다.
# 수많은 가중치가 있는데 반해 데이터가 너무 적기 때문이다.
# 데이터셋을 충분히 늘려야 성능이 좋아질것이다.

In [ ]:
# 테스트 데이터셋을 이용하여 모델의 예측 결과를 알아보기 위한 함수를 정의한다
# test_dataset의 모든 데이터에 대해 모델이 예측한 이미지 원본, 정답 label값, 확률값을 반환한다.
def get_predictions(model, iterator):
  model.eval()
  images = []
  labels = []
  probs = []
  with torch.no_grad():
    for (x, y) in iterator:
      x, y = x.to(device), y.to(device)
      y_pred, _ = model(x)
      y_prob = F.softmax(y_pred, dim=1)
      top_pred = y_prob.argmax(1, keepdim = True)

      # image, label, prob모두 matplotlib에서 사용될 것이기 때문에 cpu로 옮겨놓는다
      images.append(x.cpu())
      labels.append(y.cpu()) 
      probs.append(y_prob.cpu())


  images = torch.cat(images, dim=0)
  labels = torch.cat(labels, dim=0)
  probs = torch.cat(probs, dim=0)

  return images, labels, probs

In [ ]:
images, labels, probs = get_predictions(model, test_iterator)
# 차례대로 이미지 원본, 해당 이미지의 정답 레이블, 모델이 예측한 확률값이다.
pred_labels = torch.argmax(probs, 1)
corrects = torch.eq(labels, pred_labels)
correct_examples = []

# 모델이 정답을 맞춘 이미지를 저장한다
for image, label, prob, correct in zip(images, labels, probs, corrects):
  if correct:
    correct_examples.append((image, label, prob))

correct_example.sort(reverse=True, key=lambda x:torch.max(x[2], dim=0).values)